Importa bibliotecas, **frameworks** e arquivos .py<br>
keras: rede neural<br>
numpy: estrutura de dados<br>
pandas: estrutura de dados<br>
sklearn: normalização de dados


In [9]:
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras import optimizers
from keras.models import load_model
from keras.models import model_from_json

np.random.seed(7)

%run Python/simulation.py
%run Python/dataNumpy.py
import sys

from sklearn.preprocessing import Normalizer


In [10]:
headerInput = ['Hora', 'Temperature', 'Wind_Speed', 'Wind_Direction', 'Humidity', 'Air_Density', 'Rain_Status', 'Solteiro_PMV', 'Solteiro_Temp', 'Circulacao_PMV', 'Circulacao_Temp']
headerOutput = ['JanelaSolteiroNorte', 'PortaSolteiroSul', 'JanelaSolteiroSul']

print(len(headerInput))
print(len(headerOutput))

11
3


In [2]:
headerInput = ['Temperature', 'Wind_Speed', 'Wind_Direction', 'Humidity', 'Air_Density', 'Jardim_PMV','Jardim_Temp','Sala_PMV','Sala_Temp','Mezanino_PMV','Mezanino_Temp']
headerOutput = ['PortaSalaNorte','JanelaSalaNorte','PortaSalaLeste','JanelaSalaSul','JanelaSalaOeste','PortaCozinhaLeste','JanelaCozinhaSul','JanelaMezaninoNorte','JanelaMezaninoSul','JanelaMezaninoOeste']

Carrega dataset e gera um objeto **normalizador**

In [11]:
aux_1 = pd.read_csv("Selected/d05_input.csv")

aux_2 = aux_1[headerInput]

scaler_in = Normalizer().fit(aux_2)
aux_3 = scaler_in.transform(aux_2)

Função que permite alterar a PORTA da conexão TCP entre o energy plus, matlab e bcvtb

In [12]:
def changePort(PORT):
    base = open('simulateAndExit_base.m', 'r')
    f = open('simulateAndExit.m', 'w')
    lines = base.readlines()
    lines[37] = str(lines[37]).replace('50001', str(PORT))
    with open('simulateAndExit.m', 'w') as file_handler:
        for item in lines:
            file_handler.write("{}".format(item))
            
    base.close()
    f.close()

Altera a porta da conexão TCP <br>
Define o arquivo de saida da rede <br>

In [51]:
input_layer = len(headerInput)
output_layer = len(headerOutput)
dataset = '10'

ID = "013"
PORT = int("50"+ID)
changePort(PORT)


topology = '10'

filePath = 'Resultados/'
fileName = 'RN_d' + dataset + '_t' + topology 

**Carrega** o modelo e os pesos de **rede neural** já testada

In [52]:
# load json and create model
json_file = open('Topologias/RN_d' + dataset + '_t'+ topology +'_modelo.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("Topologias/RN_d" + dataset + "_t" + topology +"_pesos.h5")
print("Loaded model from disk")

sgd = optimizers.SGD(lr=0.1, decay=1e-5, momentum=0.3, nesterov=True)
loaded_model.compile(optimizer=sgd, loss='mean_squared_error', metrics=['mae', 'acc'])
loaded_model.summary()

print "Compilado"

IOError: [Errno 2] No such file or directory: 'Topologias/RN_d10_t10_modelo.json'

In [ ]:
## SIMULACAO
Simulate = Connection(PORT)
connec = Simulate.start()

NP_data = np.zeros((1, input_layer + output_layer))
NP_saida = np.zeros((1, output_layer))

while True:
    data = connec[0].recv(1024)
    #print ("raw data:")
    #print (data)
    
    if not data:
        break

    #dados direto do matlab em string
    fromMatlab = data.decode("utf-8")

    #lista com
    dataAndTime = fromMatlab.split(",")

    weatherInText = dataAndTime[0].replace("[", "")
    weatherInText = weatherInText.replace("]", "")

    timeInfo = myTime.getTimeInfo(int(dataAndTime[1]))


    #Gera uma lista com os dados climatologicos
    weather = []
    for x in weatherInText.split(" "):
        weather.append(float(x))

    ##        Hora, TempExterna,
    """
    'Hora',
    'TempExterna', 'ArVeloc', 'ArDirecao', 'Umidade', 'ArDensidade', 'StatusChuva',
    'PMV_JardimInverno', 'Temp_JardimInverno',
    'PMV_SalaCozinha', 'Temp_SalaCozinha',
    'PMV_Mezanino', 'Temp_Mezanino'
    """
    
    """
    print "timeInfo:"
    print timeInfo
    
    print "weather: " + str(len(weather))
    print weather[19]
    """
    
    inputRN = pd.DataFrame([[timeInfo[2], weather[0], weather[1], weather[2], weather[3], weather[7],
                weather[8], weather[21], weather[22], weather[13], weather[14]]], columns=headerInput)
    
    inputRN_np = np.array(inputRN)
       
    np_inputRN = scaler_in.transform(inputRN_np)
    
    #print "np_inputRN:"
    #print np_inputRN
   
    #np_outputRN = model.predict_on_batch(np_inputRN)
    np_outputRN = np.around(loaded_model.predict(np_inputRN, verbose=0), 2)
    
    #np_outputRN = np_outputRN, 2) #ARREDONDA
    

    
    arred = np.around(np_outputRN[0], 2)
    
    novo = arred.tolist()
    
    novo = [novo[0], 0, novo[2]]
    
    np_outputRN = np.array([novo])
    
    #print np_outputRN.shape
    
    
    np_timeInfo = np.array([timeInfo[2]])
    np_weather = np.array([weather])
    
    
    

    np_save = np.column_stack((inputRN_np, np_outputRN))
    
    
    sendList = np_outputRN.tolist()[0]

    
    print np_outputRN
    
    
    NP_data = np.row_stack((NP_data, np_save))
    NP_saida = np.row_stack((NP_saida, np_outputRN))
    
    #l = pd.Series(np_save[0,:], index = Input + Output)

    
    sendToMatlab = ', '.join(str(float("%.2f" % e)) for e in sendList )

    #print("OUTPUT DATA:")
    #print (sendToMatlab)
    connec[0].sendall(str.encode(sendToMatlab))

   


Simulate.close()

arquivo = pd.DataFrame(NP_data)
arquivo.columns=headerInput+headerOutput
arquivo = arquivo[2:]
arquivo = arquivo[:-1]
arquivo.to_csv(filePath + fileName + ".csv", index=False)


saida = pd.DataFrame(NP_saida)
saida.columns=headerOutput
saida = saida[2:]
saida = saida[:-1]
saida.to_csv(filePath + fileName + "_saida.csv", index=False)



print ("FIM!")

PYTHON || simulacao.py: Iniciando simulacao
PYTHON || simulacao.py: Esperando inicializacao do BCVTB
PYTHON || simulation.py: Recebendo dados
[[ 0.50999999  0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.47999999]]
[[ 0.5         0.          0.47999999]]
[[ 0.5         0.          0.47999999]]
[[ 0.5         0.          0.47999999]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.50999999  0. 

[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]


[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.47999999]]


[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]


[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.47999999]]
[[ 0.5         0.          0.47999999]]
[[ 0.5         0.          0.47999999]]
[[ 0.5         0.          0.47999999]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.47999999]]
[[ 0.5         0.          0.47999999]]
[[ 0.5         0.          0.47999999]]
[[ 0.5         0.          0.47999999]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]


...

[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]


[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]


[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]


[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]


[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.50999999  0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
[[ 0.5         0.          0.49000001]]
